In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf

Nhat Customer Class Implementation

In [2]:
class Customer:

    def __init__(self, file_path):
        self.file_path = file_path
        self.appearance = None
        self.columns = []

    def process_excel_file(self, source_column_name, date_column_name, amount_column_name):
        # Read the file in
        df = pd.read_excel(self.file_path)

        # Decide the time window
        min_date = min(df[date_column_name])
        max_date = max(df[date_column_name])
        dates = pd.date_range(min_date, max_date)
        # Make dataframe have row from start date to end date
        appearance_df = pd.DataFrame(index=dates)
        # Check for each unique transaction description
        for value in df[source_column_name].unique():
            value_appearances = []
            # Check for each date in the time window
            for date in dates:
                # Show row have that specific transaction date and transaction name and count
                count = len(df[(df[date_column_name] == date) & (df[source_column_name] == value)])
                # We append number of time it appears for that transaction description on that date and repeat that for all the day
                value_appearances.append(count)
            # We then set value of the specific transaction description on all the day the number of times it appears
            appearance_df[value] = value_appearances

            # Store list of column names into attribute columns for customer object
            self.columns.append(value)

        # Store appearance_df into the attribute appearance for customer object
        self.appearance = appearance_df

In [76]:
# Example usage:
# Initilize file_path and column_name
file_path = "/content/Sample Ledger.xlsx"
source_column_name = "Transaction Description"
date_column_name = "Transaction Date"
amount_column_name = "Transaction Amount"


# Create an object Customer with the parameter file path
customer = Customer(file_path)

# Call function excel_file(column_name) from object Customer
customer.process_excel_file(source_column_name, date_column_name, amount_column_name)

data_source = customer.appearance.copy()
appearance_pd = customer.appearance.copy()
columns = customer.columns

appearance_pd


,Deposit from PAYPAL from ANIL VENKATESH TRANSFER,Bill payment to State Renaissance LLC,Deposit from C106956 PROFVAL DIR DEP,Withdrawal from CHASE CREDIT CRD EPAY,Check Deposit (Mobile),Deposit from ADELPHI UNIVERSI PAYROLL,Withdrawal from AMEX EPAYMENT ACH PMT,Withdrawal from VENMO PAYMENT,Withdrawal from BK OF AMER VISA ONLINE PMT,Withdrawal from CITI AUTOPAY PAYMENT,Monthly Savings - Withdrawal to 360 Performance Savings XXXXXXX7616,Monthly Interest Paid,Withdrawal from VERIZON PAYMENTREC,Zelle money received from YA-HSIU YEN,Zelle money received from SACHIN SHAH,Withdrawal from CON ED OF NY CECONY,ATM Withdrawal - CVS STORE #0-V08 VV082672 BROOKLYN NY,Withdrawal from NYS DTF PIT Tax Paymnt,Deposit from Square Inc Payroll,ATM Withdrawal - CVS STORE #1-C08 VC080086 BROOKLYN NY
2023-11-27,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2023-11-28,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2023-11-29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2023-11-30,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
2023-12-01,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-16,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2024-02-17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2024-02-18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2024-02-19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Roya Helper Class Implementation


In [99]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf

class HelperFunctions:
   # Input data frame in order:
   # transaction_date	transaction_amount	source

  def get_periods1(data: pd.DataFrame, source_columns: list = [], window: int = 3, rollingType: str = "sliding")-> pd.DataFrame:

    output = pd.DataFrame()
    if len(source_columns)==0:
      source_columns = list(data.columns)

    period = []
    auto = []

    if rollingType == "sliding":
      data = data.rolling(3, min_periods =1).sum()

    output['Count'] = data.sum()

    try:
      for i in source_columns:

        if data[i].sum()<2:
          period.append(0)
          auto.append(0)

        else:

          start = list(data[i]).index(1)
          lags = (data.index.max()-data.iloc[start:,].index.min()).days

          period.append(pd.Series(acf(data[i].iloc[start:], nlags = lags)).iloc[3:].idxmax())
          auto.append(pd.Series(acf(data[i].iloc[start:], nlags = lags)).iloc[3:].max())

    except:
          print(i + 'might not be a source.')

    output['Period'] = period
    output['Autocorrelation'] = auto


    return output


  def get_periods(data: pd.DataFrame, source_columns: list = [], window: int = 3, rollingType: str = "sliding")-> pd.DataFrame:
    # Function to get the sliding window of a source
    # You can change rollingType to "static"

    data_in_function = data.copy()
    # DataFrame were we're storing max autocorr and the period
    output_periods = pd.DataFrame()

    # Sort values in cronological order
    data_in_function = data.sort_values(by='transaction_date', ascending = True)

    # Set date as index to use resample
    data_in_function.set_index('transaction_date', inplace = True)

    # Stores the periods of the sources
    index_corr = []

    # Stores the autocorrelation
    autocorr = []

    # Stores the sources
    source_cols = []

    # List that stores the amount of transactions of the same source
    count = []

    # Rolling sum
    roll = []

    # Checks if there were any sources past as arguments
    # If not just adds all the sources to source_columns
    if len(source_columns)==0:
      source_columns = list(data['source'].unique())

    # Loop over all the sources passed as arguments
    for source in source_columns:

      # Check if sources passed as arguments are actual transaction names
      try:

        # Creates a dataframe with only the current source
        current_source = data_in_function[data_in_function['source']==source]

        # Checks the time between the first and
        #last transaction of the actual source
        max_lags = (current_source.index.max()-current_source.index.min()).days

        # Check if the current source happened more than twice
        # If not the autocorrelation and period are set to 0
        if (len(current_source.resample('D').count()) < 3):
          autocorr.append(0)
          index_corr.append(0)
          roll.append(np.nan)

        else:

          # Calculates the rolling sum of the sliding sum
          if rollingType == "sliding":
            rolling = current_source['source'].resample('D').count().rolling(3, min_periods =1).sum()

          else:
            rolling = current_source['source'].resample('D').count().drop(columns = ['source'])

          roll.append(list(rolling))
          # Store the period of current source
          index_max_cor = pd.Series(acf(rolling, nlags = max_lags)).iloc[3:].idxmax()

          # Store autocorrelation of current source
          max_cor = pd.Series(acf(rolling, nlags = max_lags)).iloc[3:].max()

          # Adds the values to corresponding lists
          index_corr.append(index_max_cor)
          autocorr.append(max_cor)

        source_cols.append(source)
        count.append(current_source.count().max())
      # Exception raised if source is not in the bank statement as a transaction
      except:
          print(source + 'might not be a source.')

    # Prepare the data frame to return
    # Set the index as the sources passed as arguments
    output_periods.index = source_cols

    # Stores the periods in the DataFrame
    output_periods['Periods'] = index_corr

    # Stores the autocorrelations in the DataFrame
    output_periods['Autocorrelation'] = autocorr

    output_periods['Count'] = count

    output_periods['Rolling'] = roll

    # Returns the DataFrame with the periods and autocorrelations
    return output_periods

  #function to plot autocorrelation of a source
  #Parameters: df displaying cadence and source type as a str
  #Returns a plot of the autocorrelation for that source with the sliding/static window
  def visualize_acf(df: pd.DataFrame, source: str):
      try:
          # Get rolling from dataframe
          data = pd.Series(df.loc[source]['Rolling'])

          # Get Maximum Lag length
          lagLength = len(data)-1

          # Plot the autocorrelation
          plot_acf(data, lags = lagLength)
          plt.title(f"Autocorrelation for {source}")
          plt.show()
      except KeyError:
          print(f'{source} is not in Dataframe')

  # Function that gets a data frame and returns the transaction statistics
  # For the time between transactions: mean, std, and coefficient of variance
  # For the transaction amount for a source: mean, std, and coefficient of variance
  # And number of transactions of a specific source
  # Input data frame in order:
  # transaction_date	transaction_amount	source
  def get_statistics(data: pd.DataFrame, source_columns: list = []) -> pd.DataFrame:

    # store input data frame
    data_in_function = data.copy()

    # Checks if there were any sources past as arguments
    # If not just adds all the sources to source_columns
    if len(source_columns)==0:
      source_columns = list(data_in_function['source'].unique())

    # Retrieves the sources passed as arguments
    data_in_function = data.loc[data['source'].isin(source_columns)].copy()

    # creates a time difference column from the previous trasaction and the current transaction of a source
    data_in_function['time_diff'] = data_in_function.groupby(['source'])['transaction_date'].diff(periods=-1).dt.days

    # Calculates mean and std of the time difference for each soruce
    data_on_time = data_in_function.groupby('source')['time_diff'].agg(['mean','std'])

    # Calculates coefficient of variance for the time difference
    data_on_time['cv'] = (data_on_time['std']/data_on_time['mean']).abs()

    # Fills Nas with infinity
    data_on_time.fillna(np.inf, inplace = True)

    # Variable that establishes how strong is the 0,infinity -> 0,1 transform
    k = 1.5

    # Transforms scale of data
    data_on_time['kadence'] = 1-np.exp(-k*data_on_time['cv'])

    # Drops coefficient of variance column because only need the transformed version
    data_on_time.drop(columns = ['cv'], inplace = True)

    # Renames columns
    data_on_time.columns = ['Mean_Periods', 'STD_Periods', 'Kadence']

    # Calculates the mean and std of the transaction amount per source
    data_on_trans_amount = data_in_function.drop(columns = ['time_diff']).groupby('source')['transaction_amount'].agg(['count','mean','std'])

    # Calculates the coefficient of variance
    data_on_trans_amount['cv'] = (data_on_trans_amount['std']/data_on_trans_amount['mean']).abs()

    # Fills NAs with np.inf
    data_on_trans_amount.fillna(np.inf, inplace = True)

    # Variable that establishes how strong is the 0,infinity -> 0,1 transform
    k = 1.5

    # Transforms scale of coefficient of variance
    data_on_trans_amount['kadence'] = 1-np.exp(-k*data_on_trans_amount['cv'])

    # Drops coefficient of variance column because only need the transformed version
    data_on_trans_amount.drop(columns = ['cv'], inplace = True)

    # Drops columns
    data_on_trans_amount.columns = ['Count', 'Trasncation_Amount_Mean', 'Transaction_Amount_STD', 'Coefficient_of_Variance_Dollar']

    # Merges date and dollar data frames
    output_stats = data_on_trans_amount.merge(data_on_time, left_index=True, right_index= True)

    # Returns the dataset
    return output_stats